# Scan

In [ ]:
import json
from datetime import datetime
import pandas as pd
import pprint as pp
import requests, urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from tenable_authen import TenableAuthentication

In [ ]:
tenableAuthentication = TenableAuthentication()
HOSTNAME = tenableAuthentication.getHostname()
HEADER = tenableAuthentication.getRequestHeader()

## 1. Get All of Scans

In [ ]:
def getAllScan():
    getAllScan = HOSTNAME + "/rest/scan"
    allScans = requests.get(getAllScan, headers=HEADER, verify=False)
    allScans = allScans.json()["response"]
    print(len(allScans["usable"]), len(allScans["manageable"]))
    return allScans

In [ ]:
allScans = getAllScan()

In [ ]:
pd.DataFrame(allScans["usable"])

## 2. Search Scan

- *id
- **name
- **description
- **status
- ipList
- type
- plugin
- repository
- owner1
- reports
- assets
- credentials
- schedule
- policy
- createdTime
- modifiedTime

In [ ]:
def searchScan(query={}):
    searchScanURL = HOSTNAME + "/rest/scan"
    searchScan = requests.get(searchScanURL, params=query, headers=HEADER, verify=False)
    return searchScan.json()["response"]

In [ ]:
query = {
    "id" : "111"
}

In [ ]:
searchScan = searchScan(query=query)

In [ ]:
searchScan

## 3. Get Specific Scan by ScanID

In [ ]:
def getScanByScanID(scanID=""):
    getScanByScanIDURL = HOSTNAME + "/rest/scan/" + str(scanID)
    scan = requests.get(getScanByScanIDURL, headers=HEADER, verify=False)
    return scan.json()["response"]

In [ ]:
scanID = 111

In [ ]:
scan = getScanByScanID(scanID=scanID)

In [ ]:
pp.pprint(scan)

## 4. Create Scan

In [ ]:
# {
#     "name" : <string>,
#     "type" : <string> DEFAULT "policy",
#     "description" : <string> DEFAULT "",
#     "repository" : {
#         "id" : <number>
#     },
#     "zone" : {
#         "id" : <number> DEFAULT "0" (All Zones)
#     },
#     "dhcpTracking" : <string> DEFAULT "false",
#     "classifyMitigatedAge" : <number> DEFAULT "0",
#     "schedule" : {
#         "type" : "dependent" | "ical" | "never" | "rollover" | "template" <string> DEFAULT "template"
#     },
#     "reports" : [
#         {
#             "id" : <number>,
#             "reportSource" : <string> "cumulative" | "patched" | "individual" | "lce" | "archive" | "mobile"
#         }...
#     ] DEFAULT [],
#     "assets" : [
#         {
#             "id" : <number>
#         }...
#     ] DEFAULT [],
#     "credentials" : [
#         {
#             "id" : <number>
#         }...
#     ] DEFAULT [],
#     "emailOnLaunch" : <string> "false" | "true" DEFAULT "false",
#     "emailOnFinish" : <string> "false" | "true" DEFAULT "false",
#     "timeoutAction" : <string> "discard" | "import" | "rollover" DEFAULT "import",
#     "scanningVirtualHosts" : <string> "false" | "true" DEFAULT "false",
#     "rolloverType" : <string> "nextDay" | "template" DEFAULT "template",
#     "ipList" : <string> DEFAULT "" (valid IP),
#     "maxScanTime" : <number> DEFAULT "3600"
# }

In [ ]:
def createScan(payload={}):
    getURL = HOSTNAME + "/rest/scan/"
    newscan = requests.post(getURL, data=json.dumps(payload), headers=HEADER, verify=False)
    return newscan.json()

In [ ]:
scanName = ""
description = ""
repositoryID = 1
zoneID = 1
credentialID = 1000000
policyID = 1000111
ipLists = "192.168.1.1"

In [ ]:
nowTime = datetime.now()
#nowTime.strftime("%y%m%dT%H")

if nowTime.minute >= 31 and nowTime.minute <= 60:
    nowTime = nowTime.strftime("%Y%m%dT") + str(int(nowTime.hour) + 1) + "0000"
elif nowTime.minute >= 0 and nowTime.minute <= 30:
    nowTime = nowTime.strftime("%Y%m%dT%H") + "3000"
nowTime

In [ ]:
payload = {
    'name': scanName,
    'description': description,
    'repository': {
        'id': repositoryID
    },
    'schedule': {
        'start': 'TZID=Asia/Bangkok:' + nowTime,
        'repeatRule': 'FREQ=TEMPLATE;INTERVAL=1',
        'type': 'template',
        'enabled': 'true'
    },
    'dhcpTracking': 'true',
    'emailOnLaunch': 'false',
    'emailOnFinish': 'false',
    'type': 'policy',
    'policy': {
        'id': policyID
    },
    'zone': {
        'id': zoneID
    },
    'timeoutAction': 'rollover',
    'rolloverType': 'template',
    'scanningVirtualHosts': 'false',
    'classifyMitigatedAge': 0,
    'ipList': ipLists,
    'credentials': [
        {
            'id': credentialID
        }
    ],
    'maxScanTime': 'unlimited'
}
payload

In [ ]:
newscan = createScan(payload=payload)

In [ ]:
newscan

## 5. Start Specific Scan by ScanID

In [ ]:
scanid = 111

In [ ]:
getURL = HOSTNAME + "/rest/scan/" + str(scanid) + "/launch"
scan = requests.post(getURL, headers=HEADER, verify=False)
scan = scan.json()["response"]

In [ ]:
scan